In [4]:
# Imports

import os
import polars as pl
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from datetime import datetime
from math import floor
import polars as pl
from fpdf import FPDF

ModuleNotFoundError: No module named 'fpdf'

In [ ]:
# Konstanter

LIMIT_AXLES_SAME_GROUP = 1.8
AXLE_WEIGHT = 'AxleWeight'
AXLE_DISTANCE = 'AxleDistance'
LIMIT_AXLE_WEIGHT = 30 # akselvekt over 30 tonn ansees som anomalier of filtreres vekk

In [ ]:
def calculate_axle_load_distribution(filepath: str) -> tuple:

    single_axle = np.array([])
    boggi_axle = np.array([])
    triple_axle = np.array([])

    df = pl.read_csv(filepath, separator=';', truncate_ragged_lines=True, skip_rows=6, ignore_errors=True)
    df = df.filter(pl.col('GrossWeight') > 3500)
    df = df.with_columns(
        pl.col("StartTime").cast(pl.Datetime(time_unit='ms')).alias('unix_timestamp')
    )

    earliest_date = df['unix_timestamp'].min()
    latest_date = df['unix_timestamp'].max()

    for row in df.iter_rows(named=True):
        single, boggi, triple = calculate_axle_load_distribution_vehicle(row)
        single_axle = np.concatenate((single_axle, single))
        boggi_axle = np.concatenate((boggi_axle, boggi))
        triple_axle = np.concatenate((triple_axle, triple))

    return single_axle, boggi_axle, triple_axle, earliest_date, latest_date

def calculate_axle_load_distribution_vehicle(row: tuple) -> tuple:
    single_axle = np.array([])
    boggi_axle = np.array([])
    triple_axle = np.array([])

    def distribute_weight(axles_in_group, weight_in_group):
        nonlocal single_axle, boggi_axle, triple_axle
        if axles_in_group == 1:
            single_axle = np.append(single_axle, weight_in_group)
        elif axles_in_group == 2:
            boggi_axle = np.append(boggi_axle, weight_in_group)
        elif axles_in_group == 3:
            triple_axle = np.append(triple_axle, weight_in_group)

    def axle_information(row: tuple, axle: int) -> tuple:
        try:
            has_axle_syntax_1 = f'{AXLE_DISTANCE}{axle}' in row and row[f'{AXLE_DISTANCE}{axle}'] != None
            has_axle_syntax_2 = f'{AXLE_DISTANCE} {axle}' in row and row[f'{AXLE_DISTANCE} {axle}'] != None

            if has_axle_syntax_1:
                return True, float(row[f'{AXLE_DISTANCE}{axle}']), float(row[f'{AXLE_WEIGHT}{axle}']) / 1000
            elif has_axle_syntax_2:
                return True, float(row[f'{AXLE_DISTANCE} {axle}']), float(row[f'{AXLE_WEIGHT} {axle}']) / 1000
        except:
            # Støter på problemer her med at csv filene tolker data som datoer, får opp typ apr.33 etc,
            # velger dermed å droppe kranglete akseldata
            pass 

        return False, None, None

    axle = 1
    axles_in_group = 0
    weight_in_group = 0

    while True:
        has_axle, distance_from_previous_axle, weight_axle = axle_information(row, axle)
        if not has_axle:
            break

        if distance_from_previous_axle <= LIMIT_AXLES_SAME_GROUP:
            axles_in_group += 1
            weight_in_group += weight_axle

        else:
            distribute_weight(axles_in_group, weight_in_group)
                
            axles_in_group = 1
            weight_in_group = weight_axle

        axle += 1

    distribute_weight(axles_in_group, weight_in_group)
        
    return single_axle, boggi_axle, triple_axle

def extract_location(filepath):
    if 'Aanestad' in filepath:
        return 'Ånestad'
    elif 'Øysand' in filepath:
        return 'Øysand'
    elif 'Skibotn' in filepath:
        return 'Skibotn'
    elif 'Verdal' in filepath:
        return 'Verdal'



In [ ]:
datasets = (
    '../WIM-data/Kistler_Øysand/20160808-31_Kistler Øysand_4913151-export(1).csv',
    '../WIM-data/Kistler_Øysand/20160901-30_Kistler Øysand_4913151-export(2).csv',
    '../WIM-data/Kistler_Øysand/20161001-31_Kistler Øysand_4913151-export(3)-fixed.csv',
    '../WIM-data/Kistler_Øysand/20161101-30_Kistler Øysand_4913151-export(4).csv',
    '../WIM-data/Kistler_Øysand/20161201-31_Kistler Øysand_4913151-export(5).csv',
    '../WIM-data/Kistler_Øysand/20170101-31_Kistler Øysand_4913151-export(6).csv',
    '../WIM-data/Kistler_Øysand/20170201-28_Kistler Øysand_4913151-export(7).csv',
    '../WIM-data/Kistler_Øysand/20170301-31_Kistler Øysand_4913151-export(8).csv',
    '../WIM-data/Kistler_Øysand/20170401-05_Kistler Øysand_4913151-export(9)-fixed.csv',
    '../WIM-data/Kistler_Øysand/20180316_1.3.1_Kistler Øysand_4913151-export(24).csv',
    '../WIM-data/Kistler_Øysand/20180401-30_Kistler Øysand_4796227-export(12).csv',
    '../WIM-data/Kistler_Øysand/20180501-31(21-26)_Kistler Øysand_4796227-export(13).csv',
    '../WIM-data/Kistler_Øysand/20180601-30(11-30)_Kistler Øysand_4796227-export(14).csv',
    '../WIM-data/Kistler_Øysand/20180701-31(01-11)_Kistler Øysand_4796227-export(15).csv',
    '../WIM-data/Kistler_Øysand/20180801-31(10-31)_Kistler Øysand_4796227-export(16).csv',
    '../WIM-data/Kistler_Øysand/20180901-30_Kistler Øysand 4796227-export(17).csv',
    '../WIM-data/Kistler_Skibotn/combinedFiles_E8_2018_kalibrert_4okt.csv',
    '../WIM-data/Kistler_Skibotn/combinedFiles_E8_2019.csv',
    '../WIM-data/Kistler_Skibotn/combinedFiles_E8_2020.csv',
    '../WIM-data/Kistler_Verdal/20150513-20150531_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20150601-20150630_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20150701-20150731_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20150801-20150831_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20150901-20150930_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20151001-20151031_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20151101-20151130_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20151201-20151231_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20160101-20160131_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20160201-20160229_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20160301-20160331_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20160401-20160430_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Verdal/20160501-20160531_Kistler Verdal 4796227.csv',
    '../WIM-data/Kistler_Aanestad/20221014-20 Kistler_R3_ostg.csv',
    '../WIM-data/Kistler_Aanestad/20221014-20 Kistler_R3_vestg.csv',
    '../WIM-data/Kistler_Aanestad/20231001-20240123_Aanestad_Ostgående.csv',
    '../WIM-data/Kistler_Aanestad/20231001-20240123_Aanestad_Vestgående.csv'
)

In [ ]:
n_rows = len(datasets)
n_cols = 3
data_information = ('Enkeltaksler', 'Boggiaksler', 'Trippelaksler')

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4.5 * n_rows))
axes = axes.flatten()
ax_index = -1

cols = ['location', 'axlegroup', 'startdate', 'enddate', 'mean', 'median'] + [f'{i}-{i+1} tonn' for i in range(30)]
rows = []

for dataset in datasets:
    try:
        name_dataset = dataset.split('/')[-1]
        location = extract_location(dataset)
        *axle_load_distribution, earliest_date, latest_date = calculate_axle_load_distribution(dataset)
        earliest_date = earliest_date.date().isoformat()
        latest_date = latest_date.date().isoformat()

        for i in range(0, 3):
            data = axle_load_distribution[i]
            mean = np.mean(data)
            median = np.median(data)

            bins = [0] * 30
            for datapoint in data:
                if datapoint >= LIMIT_AXLE_WEIGHT:
                    continue
                bins[floor(datapoint)] += 1
            rows.append([location, data_information[i], earliest_date, latest_date, mean, median] + bins)

            ax_index += 1
            ax = axes[ax_index]
            ax.hist(data, bins=30, color='blue', edgecolor='black')
            ax.set_xlabel('Akselvekt (tonn)')
            ax.set_ylabel('Passeringer')
            ax.set_title(f'Dataset: {name_dataset}\nAkselgruppe: {data_information[i]}', fontsize=10)
            ax.text(0.5, -0.15, f'Snitt: {mean:.2f} | Median: {median:.2f}', ha='center', va='center', transform=ax.transAxes, fontsize=10)
    
    except:
        print(dataset)

df = pl.DataFrame(schema=cols, data=rows)
df.write_csv('../resultater/aksellastfordeling.csv')

plt.tight_layout()
plt.show()

In [1]:
import polars as pl
from fpdf import FPDF

def generate_pdf(data, filename='output.pdf'):
    pdf = FPDF()
    pdf.add_page()

    # Set font
    pdf.set_font('Arial', 'B', 12)

    # Define column headers
    headers = list(data[0].keys())
    column_widths = [pdf.get_string_width(header) + 10 for header in headers]

    # Add header row
    for i, header in enumerate(headers):
        pdf.cell(column_widths[i], 10, header, 1, 0, 'C')
    pdf.ln()

    # Add rows
    pdf.set_font('Arial', '', 12)
    for row in data:
        for header in headers:
            pdf.cell(column_widths[headers.index(header)], 10, str(row.get(header, '')), 1, 0, 'C')
        pdf.ln()

    # Save the PDF
    pdf.output(filename)

def pdf(dics):
    generate_pdf(dics, 'dataframe_table.pdf')

# Your main code
df = pl.read_csv('../resultater/aksellastfordeling.csv')

for axlegroup in ['Enkeltaksler', 'Boggiaksler', 'Trippelaksler']:
    print(axlegroup)

    weight_columns = [col for col in df.columns if '-' in col]
    dfs = df.filter(pl.col('axlegroup') == axlegroup)
    dfs = dfs.groupby('location').agg([
        pl.col(col).sum().alias(col) for col in weight_columns
    ])

    column_sums = dfs.select([pl.col(c).sum().alias(c) for c in dfs.columns if (c != "location" and c != "axlegroup" and c != "startdate" and c != "enddate" and c != "mean" and c != "median")])

    # Create a new DataFrame for the sum row with a name
    sum_row_df = pl.DataFrame({
        "location": ["Total"],
        **column_sums.to_dict(False)
    })

    updated_df = dfs.vstack(sum_row_df)

    dics = []

    for row in updated_df.iter_rows(named=True):
        sted = row['location']
        sum_vekt = sum(row[vektspenn] for vektspenn in row.keys() if '-' in vektspenn)
        dic = {'Sted': sted} | {vektspenn: 100 * row[vektspenn] / sum_vekt for vektspenn in row.keys() if '-' in vektspenn}
        dics.append(dic)

    pdf(dics, axlegroup)


ModuleNotFoundError: No module named 'fpdf'